In [1]:
%reload_ext autoreload
%autoreload 2
%cd ..

/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor


In [2]:
from datetime import datetime
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f'{current_date=}')

current_date=Timestamp('2024-06-03 01:00:00')


In [3]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/760982


Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-05-06 01:00:00 to 2024-06-03 00:00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.66s) 
ts_data=                     pickup_hour  rides  pickup_location_id      pickup_ts
6501   2024-05-06 01:00:00+00:00      0                   1  1714957200000
36862  2024-05-06 02:00:00+00:00      0                   1  1714960800000
107919 2024-05-06 03:00:00+00:00      0                   1  1714964400000
77124  2024-05-06 04:00:00+00:00      0                   1  1714968000000
92468  2024-05-06 05:00:00+00:00      0                   1  1714971600000
...                          ...    ...                 ...            ...
3439   2024-06-02 20:00:00+00:00      4                 265  1717358400000
3462   2024-06-02 21:00:00+00:00      1                 265  1717362000000
3973   2024-06-02 22:00:00+00:00      2                 265  1717365600000
4097   2024-06-02 23:00:00+00:

In [4]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/760982
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.9467172290342805, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9467172290342805
[LightGBM] [Warning] bagging_fraction is set=0.33824497508778884, subsample=1.0 will be ignored. Current value: bagging_fraction=0.33824497508778884


In [5]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,18.0,2024-06-03 01:00:00
1,2,15.0,2024-06-03 01:00:00
2,3,15.0,2024-06-03 01:00:00
3,4,19.0,2024-06-03 01:00:00
4,5,14.0,2024-06-03 01:00:00
...,...,...,...
260,261,13.0,2024-06-03 01:00:00
261,262,11.0,2024-06-03 01:00:00
262,263,14.0,2024-06-03 01:00:00
263,264,14.0,2024-06-03 01:00:00


In [8]:
from src.feature_store_api import get_feature_store
import src.config as config

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated by our production model",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/760982
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
feature_group.insert(predictions, write_options={'wait_for_job': True})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/760982/fs/756805/fg/873817


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/760982/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x13985fbe0>, None)